# ConvMixer

ConvMixer 架构由 [Pathes Are All You Need?](https://arxiv.org/abs/2201.09792) 提出，文章的另一个题目叫 Convolutions Attention MLPs

使用 Depthwise Convolution 来模拟 SelfAttention 在空间上进行不同位置上的特征融合，使用 Pointwise Convolution 来模拟 FFN 进行通道维度上的信息融合分析。

<div class="alert alert-info">
**摘要**：尽管卷积网络多年来一直是视觉任务中的主流架构，最近的实验表明，基于Transformer的模型，尤其是Vision Transformer (ViT)，在某些情况下可能超过其性能。然而，由于Transformer中的自注意力层具有二次运行时复杂性，ViT需要使用patch嵌入，将图像的小区域分组为单个输入特征，以适用于较大的图像尺寸。这引发了一个问题：ViT的性能是由于Transformer架构本身更强大，还是部分归因于使用patch作为输入表示？本文为后者提供了一些证据：我们提出了ConvMixer，这是一种极其简单的模型，其原理与ViT及更基础的MLP-Mixer相似，直接以patch作为输入，分离了空间和通道维度的混合，并在整个网络中保持相同的尺寸和分辨率。然而，与之不同的是，ConvMixer仅使用标准卷积来实现混合步骤。尽管其结构简单，我们的实验表明，ConvMixer在相似的参数量和数据集规模下，性能优于ViT、MLP-Mixer及其某些变体，并且还超过了经典视觉模型如ResNet。我们的代码可在 https://github.com/locuslab/convmixer 获得。
</div>

<div class="wy-nav-content-img">
    <img src="assets/ConvMixer_model_arch.png" width=960px alt="ConvMixer 模型架构示意图">
    <p>图1: ConvMixer 模型架构示意图</p>
</div>

## 代码实现

In [2]:
import copy
import torch
from torch import nn


class Residual(nn.Module):

    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return x + self.fn(x)


class ConvMixer(nn.Module):
    def __init__(self, dim, depth, kernel_size=3, patch_size=7, num_classes=1000):
        super().__init__()
        conv_mixer_layer = nn.Sequential(
            Residual(
                nn.Sequential(
                    nn.Conv2d(
                        dim, dim, kernel_size=kernel_size, groups=dim, padding="same"
                    ),
                    nn.GELU(),
                    nn.BatchNorm2d(dim),
                )
            ),
            nn.Conv2d(dim, dim, kernel_size=1),
            nn.GELU(),
            nn.BatchNorm2d(dim),
        )
        conv_mixer_layers = [copy.deepcopy(conv_mixer_layer) for i in range(depth)]
        self.model = nn.Sequential(
            nn.Conv2d(3, dim, kernel_size=patch_size, stride=patch_size),
            nn.GELU(),
            nn.BatchNorm2d(dim),
            *conv_mixer_layers,
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(dim, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [3]:
conv_mixer_model = ConvMixer(256, 8)

x = torch.randn(1, 3, 224, 224)
logits = conv_mixer_model(x)
print(logits.shape)

torch.Size([1, 1000])


## 使用 Timm 中的预训练模型

In [11]:
import timm

print(timm.list_models("convmixer*"))

['convmixer_768_32', 'convmixer_1024_20_ks9_p14', 'convmixer_1536_20']


In [13]:
from timm.models.convmixer import ConvMixer

model = ConvMixer(dim=256, depth=8, kernel_size=3, patch_size=7, num_classes=1000)
logits = model(x)
print(logits.shape)

torch.Size([1, 1000])
